In [5]:
# Import the Pinecone library
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time
from pathlib import Path
import pandas as pd
pc = Pinecone(api_key="pcsk_368Tpa_ULGYwn74G43mBVq1hEUaaReNWtM38AFjwhU3cpJjKvpXqaV4GxJDGWaY12tagMR")

In [4]:

# # Initialize a Pinecone client with your API key
# pc = Pinecone(api_key="pcsk_368Tpa_ULGYwn74G43mBVq1hEUaaReNWtM38AFjwhU3cpJjKvpXqaV4GxJDGWaY12tagMR")

# # Define a sample dataset where each item has a unique ID and piece of text
# data = [
#     {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
#     {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
#     {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
#     {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
#     {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
#     {"id": "vec6", "text": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."}
# ]

# # Convert the text into numerical vectors that Pinecone can index
# embeddings = pc.inference.embed(
#     model="multilingual-e5-large",
#     inputs=[d['text'] for d in data],
#     parameters={"input_type": "passage", "truncate": "END"}
# )

# print(embeddings)


In [9]:
# Get the folder where this script is located
df = pd.read_csv("schrute.csv", header=0)
df = df.drop("Unnamed: 0", axis=1)
print(df.columns)

df['lower'] = df['text'].str.lower()
df['lower'] = df['lower'].str.strip()
# Replace empty strings with NaN and then drop those rows
df.replace(pd.NA, "", inplace=True)

Index(['index', 'season', 'episode', 'episode_name', 'director', 'writer',
       'character', 'text', 'text_w_direction'],
      dtype='object')


In [20]:
arr = df['text'].tolist()
idx = df['index'].tolist()

In [18]:
embeddings = pc.inference.embed(
model="multilingual-e5-large",
inputs=arr[:96],
parameters={"input_type": "passage", "truncate": "END"}
)

In [19]:
# Create a serverless index
index_name = "the-office"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 

# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)


In [24]:
idx[:5]

[1, 2, 3, 4, 5]

In [26]:
df['index'].loc[2].tolist()

3

In [29]:
df['text'].loc[2]

"So you've come to the master for guidance? Is this what you're saying, grasshopper?"

In [33]:
len(embeddings)

96

In [35]:
embeddings[2]

{'values': [0.029327392578125, -0.022308349609375, ..., -0.0242156982421875, -0.000347137451171875]}

In [45]:
# Target the index where you'll store the vector embeddings
index = pc.Index("the-office")

# Prepare the records for upsert
# Each contains an 'id', the embedding 'values', and the original text as 'metadata'
records = []
for i in range(len(embeddings[:5])):
    records.append({
        "id": str(df['index'].loc[i].tolist()),
        "values": embeddings[i]['values'],
        "metadata": {'text': df['text'].loc[2]}
    })


# Upsert the records into the index
index.upsert(
    vectors=records,
    namespace="example-namespace"
)


upserted_count: 5

In [42]:
records[0]['id']

1